# Practicing Good Enough Software and Change Management Practices
 


In [14]:
!mkdir -p ~/agave

%cd ~/agave

!pip3 install setvar

import runagavecmd as r
import re
import os
import sys
import json
from setvar import *
from time import sleep

loadvar()
!auth-tokens-refresh || auth-tokens-create

/home/jovyan/agave
  Running setup.py bdist_wheel for setvar ... done
  Stored in directory: /home/jovyan/.cache/pip/wheels/c7/cc/6f/fa8b8dcd762a31d58ba0c3dd0992200b5a32eddecd18fc7d83
Successfully built setvar
You are using pip version 9.0.1, however version 10.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


ModuleNotFoundError: No module named 'runagavecmd'

## Software  



> Place a brief explanatory comment at the start of every program  


Our previous notebook walked us through the process of building and testing the FUNWAVE-TVD app a couple different ways. We can take a further step towards improving the user experience by including a static test dataset with the source code so every user can run the exact same build, test, and validation commands with confidence that the results they got were the ones they should get. This lines up favorably with the Good Enough recommendation to:  

> Provide a simple example or test data set

Let's take a moment to add the input we ran in our last example to the repository on our sandbox and add it to version control.

In [15]:
# have Agave copy our last input file over. We can get the input file from
# the details of the job we just ran.
JOB = ! jobs-search --verbose --limit=1 --filter=id,inputs  owner=${AGAVE_USERNAME}

j

# files-import -U "$DATAFILE" -S $STORAGE_SYSTEM FUNWAVE_TVD/data
r.runagavecmd('ls -al /home/jovyan/FUNWAVE_TVD/data && ' + 
              'cp $(basename $DATAFILE) /home/jovyan/FUNWAVE_TVD/data/ && ' +
              '/home/jovyan/FUNWAVE_TVD/data/ && ' + 
              'cd /home/jovyan/FUNWAVE_TVD && ' + 
              'git add -A data', 
              os.environ['DATAFILE'].trim())




NameError: name 'r' is not defined

Now that we have a sample dataset, let's update our app definition to include the sample dataset as the default input. In doing so, we can guarantee that users have a predictable experience the first time they run our app.

In [ ]:
# move the file so we can write to app.json as our ouput next
mv app.json app.json.old
# use jq to replace the value of our input field to the value of our sample dataset
NEW_DATAFILE="agave://$STORAGE_SYSTEM/$DEPLOYMENT_DIR/$(basename $DATAFILE)"
jq --arg newontology 'text' --arg newdatafile "$NEW_DATAFILE" \
    '. | .inputs[0].semantics.ontology += [ $newontology ] | .inputs[0].value.default = $newdatafile ' app.json.old > app.json
rm app.json.old

At some point you will want to publish the results of your research. At that time, you will need to provide references to your data, code, and results. The Good Enough recommendation is to: 

> Submit code to a reputable DOI-issuing repository

In the context of this tutorial, the repository is Github, which does not issue DOI. Fear not. An article from the Github blog on [Making Your Code Citeable](https://guides.github.com/activities/citable-code/) explains how you can get a free DOI for your Github repository from [Zenodo](https://zenodo.org/). [Figshare](https://figshare.org) also provides DOI for data hosted there.

One of the easiest ways to get started with zenodo is to add a [codemeta]() file to your project so it can be discovered in way that spans languages, publishing services, and dependency system syntax. We included a sample codemeta file for our FUNWAVE-TVD application to this repository. We'll copy it over to the FUNWAVE_TVD repository root directory and add it to version control. 

In [ ]:
files-upload -F $HOME/etc/codemeta.json -S $STORAGE_SYSTEM FUNWAVE_TVD
ssh sandbox "cd FUNWAVE_TVD && git add -A codemeta.json"

## Keeping Track of Changes  

Knowing how a piece of software changes over time can be challenging to do for invested iniviuals. For users, it can be downright intimidating, bordering on impossible. The Good Enough recommendation says to:  

> Create, maintain, and use a checklist for saving and sharing changes to the project  

One way to do that is by keeping a changelog. Changelogs are structued text documents that describe the major, an sometimes minor changes to a project over time an release. There are many way to stucture a changelog. For reasons we will quickly see, we recommend the format found at [keepachangelog.com](http://keepachangelog.com/). Their changelog format is a machine-parsable Markdown format which leverages semantic versioning. 

While we could generate the changelog by hand, for existing projects and active ones, this quickly becomes enough of a hassle to dissuade us from keeping up with. Luckily, we can leverage projects such as the [gitchangelog](https://github.com/vaab/gitchangelog) project to generate and maintain our changelog for us. The outcome will be a file named `CHANGELOG.md` that we can add and commit to our repository


In [1]:
# let's run the gitchangelog utility on the repository
r.runagavecmd('cd FUNWAVE-TVD && gitchangelog | tee -a CHANGELOG.md', 
             "https://raw.githubusercontent.com/agavetraining/pearc18/master/etc/.gitchangelog.rc")

bash: gitchangelog: command not found
Please run /home/jovyan/src/cli/bin/tenants-init to initialize your client before attempting to interact with the APIs.


: 1

Another Good Enough software recommendation is:

> Share changes frequently  

Having a changelog is much more useful when it is up to date. Rather than counting on ourself and our colleagues to remember to rebuild it each time, let's set up a git hook to rerun `gitchangelog` and update our changelog whenever we merge our develop branch back into master. The following script should handle that for us. 

```bash
#!/bin/sh

# post-checkout hook - looks for changes to source files and, if
# found, generates a new changelog file from the commit history.

# To install, copy to your project's .git/hooks folder, and 
# `chmod +x post-merge`

function changed {
  git diff --name-only HEAD@{2} HEAD | grep "^$1" > /dev/null 2>&1
}

gitchangelog 
```

We can install the scrirpt on our sandbox by copying the script to the repository `.git/hooks` folder and assigning execute privileges.

In [ ]:
files-upload -F scripts/git_hooks/post-merge -S $STORAGE_SYSTEM FUNWAVE-TVD/.git/hooks
ssh sandbox "chmod +x FUNWAVE-TVD/.git/hooks/post-merge"

bash: maintain,: command not found


## Manual Versioning  

The Good Enough recommenation is to:  

> Copy the entire project whenever a significant change has been made      

If your entire project, including data, is under version control, they may be a bit of overkill when you can just as easily branch the repository, however, as we are seeing, in our overall digital R&D lifcycle, there are other considerations upstream from just managing the code. 


## Maintaining your Agave app 

Keeping your Agave app in sync with the code it represents will help your users feel secure that what they think they will be running is what will actually be run. Aside from the standard revision increment that happens whenever you update your app definition, additional information can be added to better inform users of its activity, utility, and reliability. 


### Updating your app definition  

The lowest hanging fruit is simply to update your app definition. If any ontological terms, description, etc has changed, the `app.json` file should be updated to reflect the latest information. We can automate this with the use of the same `git-merge` file we used to update our changelog. 

Adding the following code to the end of that file will result in our app being updated whenever we merge into our master branch. 

```shell
#  no way 
apps-addupdate -F app.json $APP_ID  

```  


### Updating your app metadata

If you have an additional metadata such as a buildfile, yaml service definition, or CWL file for your app, updating the app's metadata with any new or changed content should be done now.  

### Mirroring your app tags

The tags you assign to your app probably will not change very often, but in the event a new feature requires changes to existing tags, or a reorganization of tags, then those updates should be done now.  

### Deprecating your old app(s)

Some releases, such as rollbacks, yanked releases, and security patches may justify the deprecation or disabling of a previous app version. This is where it would be done.

### Rerunning benchmarks

Once your build, unit, and integration tests pass, your app and its data are updated, and your assets are publised to their new locatino, benchmark jobs can be run to measure application performance before and after the release.

### Updating analytics and published data  

When your benchmarks complete, they can be published along with other app assets as metadata, archived with the benchmark jobs themselves, or saved as part of a document kept with the code.  

### Version Control Systems  

Let's just quickly acknowledge that we need to use some form of version control if we're going to talk about anything else related to proper software development in an open science community. The Good Enough recommendation says it all: 

> Use a version control system


Using a version control system does not imply that everything should go in there. While Github, GitLab, and Bitbucket have large file support now, that does not mean we should commit large binary files without proper consideration.

> Consider what not to put under version control 

As a rule of thumb, lean away from committing large binary docs and anyting that doens't lend itself well to text diffs.



Before we finish this notebook, let's go ahead and commit the changes to our repository.

In [ ]:
# remote in and commit the directory
ssh sandbox "cd FUNWAVE_TVD && git add -A data && git commit -m 'Adding example dataset' ."

If everything succeeded, you should see your version number incremented and a build job now running in your job history.